### Data Collection..!

In [ ]:
import pandas as pd
import numpy as np

messages = pd.read_csv(
    '../NLP-for-Transformers/Datum/SMSSpamCollection',
    # sep='\t',
    # names=['label', 'message']
)

messages

### Data Preparation and preprocessing..!

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer # See the Capital letters

stop_words = set(stopwords.words('english')) # List to Set
ps = PorterStemmer()

In [ ]:
corpus = [] # Group of Sentences
for i in range(len(messages)): # for each sentence in that message
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower().split()

    cleaned_words = [] # Group of Individual Words
    for word in review: # for each word in that sentence
        if word not in stop_words:
            s_word = ps.stem(word)
            cleaned_words.append(s_word) # Adds the Stemmed word to the Sentence List

    Sentence = ' '.join(cleaned_words) # Forming the sentance from Cleaned Words
    corpus.append(Sentence) # Adds the cleaned sentence to the corpus List 

corpus

#### Applying OHE for the y lable: (Creating y Here..!)

In [ ]:
y = pd.get_dummies(messages['label']) # Dummies --> OHE.
y = y.iloc[:,1] # selecting the 2nd column of the spam,ham
values

### **Step-2: Text --> Vectors** (Creating X Here..!)

### **1. Creating the Bag Of Words Model for Vectorization**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Using CountVectorizer for BoW Vectors
cv = CountVectorizer(max_features = 2500) # Implement the Bag of Words
# cv = CountVectorizer(max_features = 2500, binary = True, ngram_range = (1,2))
X = cv.fit_transform(corpus)

X = X.toarray()

In [ ]:
# Train Test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

#### **MultinomialNB**

In [ ]:
# Creating Object for the model class..!

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train, y_train)

# Prediction

y_pred = spam_detect_model.predict(X_test)

# Accuracy Metrics

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
accuracyScore = accuracy_score(y_test, y_pred)
print(accuracyScore)
classy_report = classification_report(y_test, y_pred)
print(classy_report)

#### **RandomForestClassifier**

In [ ]:
# Creating Object for the model class..!

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train) # not fit_transform

# Predicting

y_pred = rf.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(acc)
clss = classification_report(y_test, y_pred)
print(clss)

### **2. Creating the TF-IDF Model for Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Using TfidfVectorizer for TF-IDF Vectors
tv = TfidfVectorizer(max_features = 2500)
# tv = TfidfVectorizer(max_features = 2500, ngram_range = (1,2))
X = tv.fit_transform(corpus)
X.toarray()
# y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

#### **MultinomialNB**

In [ ]:
# Creating Object for the model class..!

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train, y_train)

# prediction

y_pred = spam_detect_model.predict(X_test)

# Accuracy Metrics

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
accuracyScore = accuracy_score(y_test,y_pred)
print(accuracyScore)
classy_report = classification_report(y_test,y_pred)
print(classy_report)

#### **RandomForestClassifier**

In [ ]:
# Creating Object for the model class..!

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train) # not fit_transform

# Predicting

y_pred = rf.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print(acc)
clss = classification_report(y_test, y_pred)
print(clss)

### **3. Creating the Word2Vec Model for Vectorization**

### **Loading the data or Data Collection..!**

In [ ]:
import gensim.downloader as api

wv = api.load("word2vec-google-news-300")

### Data Preprocesssing and preparation..!

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess # simple_preprocess will simply convert to lowercase..!

words=[]
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

print(len(words))
words

#### **# Creating Object for the model class..!**

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(words,window=5,min_count=2) # Here Vector_size=100 and epochs=5 are default values..!
model.wv.index_to_key

### **4. Creating the AvgWord2Vec Model for Vectorization**

#### **Defining the function for avg_word2vec**

In [ ]:

def avg_word2vec(doc):
    """
    Compute the average Word2Vec embedding for a document.
    doc: list of tokens (words)
    """
    
    vectors = []  # to store word vectors found in the model
    
    for word in doc:
        # check if word exists in Word2Vec vocabulary
        if word in model.wv.index_to_key:
            vectors.append(model.wv[word])
    
    # if no words from doc are in vocabulary, return a zero vector
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    
    # compute and return the mean vector
    avg_vector = np.mean(vectors, axis=0)
    return avg_vector

#### **Calling the avg_word2vec function**

In [ ]:
# Apply for the entire sentences

X=[]
for i in tqdm(range(len(words))):
    # print("Hello",i)
    X.append(avg_word2vec(words[i]))

X.shape

X_new = np.array(X)

In [ ]:
### Train Test Split and Apply a Model to this..!